In [1]:
#import necessary modules
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

# files
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/dataset-lowest-aic452022/dataset_lowestAIC_longlat.csv
/kaggle/input/datasetlowestaic/dataset_lowestAIC.csv
/kaggle/input/iterated-12preds/iterated_12vars.csv


In [2]:
##SOURCE: http://www.science.smith.edu/~jcrouser/SDS293/labs/lab10-py.html

In [3]:
#import dataset for modeling
df = pd.read_csv(r'C:/Users/foeke/OneDrive/Documenten/submitting paper/All scripts - paper/data/GlobalModelData/PredictingDataset.csv', sep=',')
df

In [ ]:
#remove unique idenifier & geodata
df = df.drop(['Unnamed: 0', 'Longitude', 'Latitude'], axis=1)
#replace NaN's with 0's
df = df.fillna(0)

y = df['mean_value_NO2']#specify target
x = df.drop(['mean_value_NO2'], axis=1)#predictors
feature_list = list(x.columns)

##GENERATING CROSS VALIDATION (75% TRAINING OF MODEL; 25% TESTING OF MODEL;EVALUATING MODEL PERFORMANCE)

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.25, random_state=20)
#random state is needed to ensure that same results are generated each time.

##ENSURING THAT X- AND Y-TRAINING SET CONTAIN SAME AMOUNT OF ROWS. SAME FOR TESTING.
print('Training Features Shape:',X_train.shape)
print('Training Labels Shape:', Y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', Y_test.shape)

In [8]:
## == FINETUNING ALPHA == ##

o = []
def alp(g):
    model_ridge = Ridge(alpha=g, normalize=True)
    model_ridge.fit(X_train, Y_train)
    predtrain= model_ridge.predict(X_train)
    prediction_adjusted_alpha= model_ridge.predict(X_test)
    
    ##RMSE
    print(g)

    def rmse(predictions, targets):
        return np.sqrt(((predictions - targets) ** 2).mean())

    print("testing rmse: ", rmse(prediction_adjusted_alpha, Y_test))

    ##R2
    print("R2 score testing data: ", r2_score(Y_test, prediction_adjusted_alpha))

    ##MEAN ABSOLUTE ERROR 
    print("MAE testing data: ", mean_absolute_error(Y_test, prediction_adjusted_alpha))

In [9]:
#examine performance results per alpha parameter

o = np.arange(0.1, 1.1, 0.1)
store = []
for i in o:
    alp(i)

0.1
testing rmse:  8.872044979884354
R2 score testing data:  0.5698366913178774
MAE testing data:  6.311794742501619
0.2
testing rmse:  8.862510003844855
R2 score testing data:  0.5707608058402517
MAE testing data:  6.323732701131643
0.30000000000000004
testing rmse:  8.860960745890774
R2 score testing data:  0.5709108635802935
MAE testing data:  6.337150876981456
0.4
testing rmse:  8.86534853534544
R2 score testing data:  0.5704858038165342
MAE testing data:  6.3510386047340885
0.5
testing rmse:  8.874365009729964
R2 score testing data:  0.5696116874537733
MAE testing data:  6.373593062384671
0.6
testing rmse:  8.88708961946601
R2 score testing data:  0.5683765681874638
MAE testing data:  6.399741176508758
0.7000000000000001
testing rmse:  8.902846169469385
R2 score testing data:  0.5668446999668264
MAE testing data:  6.427109429156619
0.8
testing rmse:  8.921118826795539
R2 score testing data:  0.5650648150430547
MAE testing data:  6.456982343641791
0.9
testing rmse:  8.9415013580885

In [10]:
ridge_adjusted_alpha = Ridge(alpha = 0.3, normalize = True)
ridge_adjusted_alpha.fit(X_train, Y_train) # Fit a ridge regression on the training data
predtrain = ridge_adjusted_alpha.predict(X_train)
prediction_adjusted_alpha = ridge_adjusted_alpha.predict(X_test)           # Use this model to predict the test data
print(pd.Series(ridge_adjusted_alpha.coef_, index = x.columns)) # Print coefficients


nightlight_450             0.066879
nightlight_3150            0.069768
population_1000            0.000287
population_3000            0.000048
road_class_2_100           0.006716
road_class_2_25            0.032623
road_class_2_50            0.019107
road_class_3_3000          0.000011
road_class_3_300           0.000689
trop_mean_filt_2019    77874.183155
BldDen100                  3.439256
trafBuf100                 0.003522
dtype: float64


In [11]:
##RMSE 
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

print("training rmse: ", rmse(predtrain, Y_train))
print("testing rmse: ", rmse(prediction_adjusted_alpha, Y_test))
##R2
print("R2 score training data: ", r2_score(Y_train, predtrain))
print("R2 score testing data: ", r2_score(Y_test, prediction_adjusted_alpha))
##MEAN ABSOLUTE ERROR 
print("MAE training data: ", mean_absolute_error(Y_train, predtrain))
print("MAE testing data: ", mean_absolute_error(Y_test, prediction_adjusted_alpha))

training rmse:  6.484889544439325
testing rmse:  8.860960745890774
